In [99]:
from sqlalchemy import create_engine
import pandas as pd
import json


# Dados de conexão
user = 'sa'
password = 'master'
host = 'localhost'
database = 'Serasa'

conn_str = f'mssql+pyodbc://{user}:{password}@{host}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

nomesAdicionado = set()

tipos_objeto = {
    8: "Transacao",
    9: "Subprograma"
}

try:
    def tipoChamada(chamador, chamado, linha):
        conn = engine.connect()
        query = f"select top 1 tipo from tb_chamadas where fonte = '{chamador}' and chamado = '{chamado}' and linfonte = {linha}"
        print(query)
        df = pd.read_sql(query, conn)
        conn.close()
        json_data = df.to_json(orient='records')
        return json_data


    def infoProg(programa):
        conn = engine.connect()

        query = f"select top 1 nome, ext, parent, amode, rmode from tb_programas where nome = '{programa}'"

        df = pd.read_sql(query, conn)
        
        conn.close()
        
        json_data = df.to_json(orient='records')

        return json_data

    def cadeia(subprograma):
        conn = engine.connect()

        query = f"select objeto as nome, linha, comando, tipo_objeto from RL_LABELS where TIPO_OBJETO in (8, 9) and TIPO_RELACAO = 'p' and origem like '%.%.{subprograma}' order by linhaexp"

        df = pd.read_sql(query, conn)
        
        conn.close()
        
        json_data = df.to_json(orient='records')

        data = json.loads(json_data)
        

        for obj in data:
            info = json.loads(infoProg(obj.get("nome")))
            tipo_chamada = json.loads(tipoChamada(subprograma, obj['nome'], obj['linha']))
            if len(info) > 0:
                obj['ext'] = info[0].get("ext")
                obj['amode'] = info[0].get("amode")
                obj['rmode'] = info[0].get("rmode")

            else:
                obj['ext'] = "fonte nao carregado"
            obj['tipo'] = tipos_objeto.get(obj['tipo_objeto'], "Tipo nao definido")
            obj['tipo_chamada'] = tipo_chamada[0].get("tipo") if  len(tipo_chamada) > 0 else ''
            if(obj['comando'] == 128 or obj['tipo_objeto'] == 8 or obj['nome'] in nomesAdicionado):
                obj['chamados'] = []
                if(obj['tipo_objeto'] == 8):
                    obj['ext'] = ''
            else:
                nomesAdicionado.add(obj['nome'])
                obj['chamados'] = cadeia(obj['nome'])
            obj.pop("comando")
            obj.pop("tipo_objeto")
            
            
        return data

    data = json.loads(infoProg('EFXAMCA'))
    

    for obj in data: 
        obj['chamados'] = cadeia(obj['nome'])

    json_data_com_info_estatica = json.dumps(data)

    print(json_data_com_info_estatica)    

except Exception as e:
    print(f"Erro ao conectar: {e}")


select top 1 tipo from tb_chamadas where fonte = 'EFXAMCA' and chamado = 'GR88' and linfonte = 235
Erro ao conectar: list index out of range
